In [1]:
!which chromedriver

/usr/local/bin/chromedriver


In [2]:
#import pymongo
from bs4 import BeautifulSoup as bs
import os
import requests
import shutil
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import urllib.request
import urllib.parse
import re

#Dictionary to collect data
mars = {}

In [3]:
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

### NASA Mars News

In [4]:
browser = init_browser()

# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')
    #print(soup.prettify)

In [5]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list

# Put together link to top story
link = [a['href'] for a in soup.select("a[href*=news]")][0]
base = "https://mars.nasa.gov/news"
url = urllib.parse.urljoin(base,link)

# Fetch headline and text  
browser.visit(url)
news_title = browser.find_by_tag('h1').value
news_p = browser.find_by_id('primary_column').value
news_p = news_p.replace('\n', ' ')
#tried a million ways to get rid of remaining \ (see old notebook), no luck yet
#But they are gone in the Chrome versio, thanks to whoever...
#story = {'news_title': news_title,'news_p': news_p}

# Add title and text to data dictionary 
mars["news_title"] = news_title
mars["news_p"] = news_p
  
news_title
news_p

'Did you know NASA\'s next Mars rover has a nearly identical sibling on Earth for testing? Even better, it\'s about to roll for the first time through a replica Martian landscape. As NASA\'s Mars rover Perseverance hurtles through space toward the Red Planet, the six-wheeler\'s twin is ready to roll here on Earth. A full-scale engineering version of the Mars 2020 Perseverance rover – outfitted with wheels, cameras, and powerful computers to help it drive autonomously – has just moved into its garage home at NASA\'s Jet Propulsion Laboratory in Southern California. This rover model passed its first driving test in a relatively tame warehouselike assembly room at JPL on Sept. 1. Engineers expect to take it out next week into the Mars Yard, where a field of red dirt studded with rocks and other obstacles simulates the Red Planet\'s surface. Download video › "Perseverance\'s mobility team can\'t wait to finally drive our test rover outside," said Anais Zarifian, the mobility test bed engin

### JPL Mars Space Images - Featured Image

In [6]:
# Thanks to GitHub for the fancybox help, I was really stuck. Didn't realize "fancybox" could be a class. 
#https://github.com/ZL14E161110/HW13---Web-Scraping-and-Document-Databases
#Didn't look at the rest of it, although I'm sure it's brilliant. 

#Assumes the latest Mars image will always be the one in the first position of the grid...
#Note 0 is the overall featured image, which is not necessarily of Mars.

# Visit site and make soup
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
response = requests.get(url)
soup = bs(response.text, 'lxml')

#Get image 
partial_address = soup.find_all('a', class_='fancybox')[1].get('data-fancybox-href').strip()
base = "https://www.jpl.nasa.gov"
featured_image_url = urllib.parse.urljoin(base, partial_address)
#featured_image_url

#Add url to dictionary
mars["featured_image_url"] = featured_image_url

mars

{'news_title': "NASA Readies Perseverance Mars Rover's Earthly Twin",
 'news_p': 'Did you know NASA\'s next Mars rover has a nearly identical sibling on Earth for testing? Even better, it\'s about to roll for the first time through a replica Martian landscape. As NASA\'s Mars rover Perseverance hurtles through space toward the Red Planet, the six-wheeler\'s twin is ready to roll here on Earth. A full-scale engineering version of the Mars 2020 Perseverance rover – outfitted with wheels, cameras, and powerful computers to help it drive autonomously – has just moved into its garage home at NASA\'s Jet Propulsion Laboratory in Southern California. This rover model passed its first driving test in a relatively tame warehouselike assembly room at JPL on Sept. 1. Engineers expect to take it out next week into the Mars Yard, where a field of red dirt studded with rocks and other obstacles simulates the Red Planet\'s surface. Download video › "Perseverance\'s mobility team can\'t wait to finall

### Mars Facts

In [79]:
url = 'https://space-facts.com/mars/'

In [80]:
tables = pd.read_html(url)
#tables = pd.read_html(url, header=None, index_col=0)
type(tables)

list

In [81]:
mars_facts_df = tables[0]
mars_facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [82]:
mars_facts_html_table = mars_facts_df.to_html(header=False, border=False, index=False)
mars_facts_html_table = mars_facts_html_table.replace('\n', '')
mars_facts_html_table
#print(mars_facts_html_table)

'<table border="False" class="dataframe">  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [14]:
#Add table to dictionary
mars["table"] = mars_facts_html_table
mars

{'news_title': "NASA Readies Perseverance Mars Rover's Earthly Twin",
 'news_p': 'Did you know NASA\'s next Mars rover has a nearly identical sibling on Earth for testing? Even better, it\'s about to roll for the first time through a replica Martian landscape. As NASA\'s Mars rover Perseverance hurtles through space toward the Red Planet, the six-wheeler\'s twin is ready to roll here on Earth. A full-scale engineering version of the Mars 2020 Perseverance rover – outfitted with wheels, cameras, and powerful computers to help it drive autonomously – has just moved into its garage home at NASA\'s Jet Propulsion Laboratory in Southern California. This rover model passed its first driving test in a relatively tame warehouselike assembly room at JPL on Sept. 1. Engineers expect to take it out next week into the Mars Yard, where a field of red dirt studded with rocks and other obstacles simulates the Red Planet\'s surface. Download video › "Perseverance\'s mobility team can\'t wait to finall

### Mars Hemispheres

In [15]:
#navigate to starting page
browser=init_browser()
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [16]:
#Make soup
# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')
#print(soup.prettify)

In [17]:
#Find titles and links for full-scale images

#title_list = []
#img_url_list = []
base = "https://astrogeology.usgs.gov"
hemisphere_image_urls = []


for link in soup.find_all(class_='itemLink product-item'):
    partial = link.get('href')    
    thumbnail_url = urllib.parse.urljoin(base, partial)
    browser.visit(thumbnail_url)
    xpath = "//a[@class='open-toggle']"
    switch = browser.find_by_xpath(xpath)
    switch.click()
    html = browser.html
    soup = bs(html, 'lxml')
    imgs = soup.find(class_='downloads')
    html = browser.html
    soup = bs(html, 'lxml')
    title = soup.find(class_ = 'title').get_text()
    img_url = [a['href'] for a in soup.select('a:contains(Original)')]
    img_url = str(*img_url)
    hemisphere_image_urls.append({'title':title, 'img_url':img_url})
browser.quit()
#hemisphere_image_urls

mars["hemisphere_image_urls"] = hemisphere_image_urls
mars

#print(image_url_list)


#    title_list.append(title)
#    image_url_list = img_url_list.append(*img_url)
#    title_list
#    print(title, *img_url)

{'news_title': "NASA Readies Perseverance Mars Rover's Earthly Twin",
 'news_p': 'Did you know NASA\'s next Mars rover has a nearly identical sibling on Earth for testing? Even better, it\'s about to roll for the first time through a replica Martian landscape. As NASA\'s Mars rover Perseverance hurtles through space toward the Red Planet, the six-wheeler\'s twin is ready to roll here on Earth. A full-scale engineering version of the Mars 2020 Perseverance rover – outfitted with wheels, cameras, and powerful computers to help it drive autonomously – has just moved into its garage home at NASA\'s Jet Propulsion Laboratory in Southern California. This rover model passed its first driving test in a relatively tame warehouselike assembly room at JPL on Sept. 1. Engineers expect to take it out next week into the Mars Yard, where a field of red dirt studded with rocks and other obstacles simulates the Red Planet\'s surface. Download video › "Perseverance\'s mobility team can\'t wait to finall

## Step 2 - MongoDB and Flask Application

Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.

* Start by converting your Jupyter notebook into a Python script called `scrape_mars.py` with a function called `scrape` that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.

* Next, create a route called `/scrape` that will call your `scrape_mars.scrape` function. Note that you'll have to import `scrape_mars.py`. 

  * Store the dictionary that gets returned to your MongoDB.

* Create an index route `/` that will query your Mongo database and pass the Mars data into an HTML template to be displayed. 

* Create a template HTML file called `index.html` that will take the dictionary of Mars data and display its values in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.

![final_app_part1.png](Images/final_app_part1.png)
![final_app_part2.png](Images/final_app_part2.png)